# 1.Required packages

In [ ]:
# Suggestion: Create new environment
# Optional
# python -m pip install --upgrade pip wheel setuptools
# Required
# python -m mlflow==2.14.3 boto3 psycopg2 cryptography
# if psycopg2 cannot install it will suggest psycopg2-binary
# python -m mlflow==2.14.3 requests python-keycloak==4.2.2 pyjwt

In [16]:
import os
import requests
import mlflow
import jwt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor


# 2. Set credentials

In [17]:
# Set credentials in empty strings
data = {
     "username": "user1",
     "password": "password1"
}

# Set a name for your experiment
experiment_name = "diabetes_demo"


# 3. Request token

In [ ]:
resp = requests.post( 
    'http://localhost:5000/',
    json= data,
    headers= {"Content-Type": "application/json"}, 
    timeout= 10
)

print( resp.status_code)
# It should return 200.

In [24]:
try:
    pass_token = jwt.decode(resp.json()['token'], "U2hvdWxkU2V0UGFzc3dvcmQ0", algorithms=["HS256"])
except jwt.ImmatureSignatureError:
    from time import sleep
    sleep(5)
    pass_token = jwt.decode(resp.json()['token'], "U2hvdWxkU2V0UGFzc3dvcmQ0", algorithms=["HS256"])


# 4. Setting env variables

In [25]:
# Don't change anything

if 'MLFLOW_TRACKING_USERNAME' in os.environ:
    del os.environ['MLFLOW_TRACKING_USERNAME']
if 'MLFLOW_TRACKING_PASSWORD' in os.environ:
    del os.environ['MLFLOW_TRACKING_PASSWORD']
os.environ['MLFLOW_TRACKING_TOKEN'] = pass_token['access_token']
os.environ["AWS_ACCESS_KEY_ID"]= pass_token['access_key_id']
os.environ["AWS_SECRET_ACCESS_KEY"]= pass_token['secret_access_key']
os.environ["AWS_SESSION_TOKEN"]= pass_token['session_token']
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5000'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'

In [ ]:
mlflow.set_experiment(experiment_name= experiment_name)

mlflow.autolog()
with mlflow.start_run() as run:

    db = load_diabetes()

    X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

    # Create and train models.
    rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)
    rf.fit(X_train, y_train)

    # Use the model to make predictions on the test dataset.
    predictions = rf.predict(X_test)
